# SQL - Understanding the bicycle market 

## 📖 Background
You work for a chain of bicycle stores. Your new team leader comes from a different industry and wants your help learning about the bicycle market. Specifically, they need to understand better the brands and categories for sale at your stores.

## 💪 Challenge II
Help your team leader understand your company's products. Include:

1. What is the most expensive item your company sells? The least expensive?
2. How many different products of each category does your company sell?
3. What are the top three brands with the highest average list price? The top three categories?
4. Any other insights you found during your analysis?

# Bicycle Market Analysis Report

## Introduction
This report provides an analysis of the bicycle market data available at a chain of bicycle stores. The aim of this report is to provide insights into the products sold at the stores, including the most and least expensive items, the number of different products in each category, and the top three brands and categories with the highest average list price. The data used in this report is available in three tables: products, brands, and categories.

## Data Preparation:


In [19]:
SELECT *
FROM products; 

SELECT *
FROM brands;

SELECT *
FROM categories;

product_id                        product_name  ...  model_year  list_price
0           1.0                     Trek 820 - 2016  ...      2016.0      379.99
1           2.0  Ritchey Timberwolf Frameset - 2016  ...      2016.0      749.99
2           3.0     Surly Wednesday Frameset - 2016  ...      2016.0      999.99
3           4.0            Trek Fuel EX 8 29 - 2016  ...      2016.0     2899.99
4           5.0        Heller Shagamaw Frame - 2016  ...      2016.0     1320.99
..          ...                                 ...  ...         ...         ...
332         NaN                                None  ...         NaN         NaN
333         NaN                                None  ...         NaN         NaN
334         NaN                                None  ...         NaN         NaN
335         NaN                                None  ...         NaN         NaN
336         NaN                                None  ...         NaN         NaN

[337 rows x 6 columns]

### 1.Most and least expensive items
The first query will select the product_id, name, and list_price columns from the products table, order the data by list_price in descending order and limit the result to 1 row. This will give us the most expensive item.
The second query is similar, but the list_price is ordered in ascending order, giving us the least expensive item.

In [20]:
SELECT product_name, list_price
FROM products
WHERE list_price = (SELECT MAX(list_price) FROM products);

SELECT product_name, list_price
FROM products
WHERE list_price = (SELECT MIN(list_price) FROM products);

product_name  list_price
0           Trek Domane SLR 9 Disc - 2018    11999.99
1  Strider Classic 12 Balance Bike - 2018       89.99

### 2.Different products in each category
This query will join the products and categories tables on the category_id column. Then, it will group the data by category_id and count the number of distinct product_ids for each category

In [21]:
SELECT category_name, COUNT(product_id) as 'Number of Products'
FROM products
JOIN categories ON products.category_id = categories.category_id
GROUP BY category_name;


category_name  Number of Products
0    Children Bicycles                  59
1     Comfort Bicycles                  30
2    Cruisers Bicycles                  78
3  Cyclocross Bicycles                  10
4       Electric Bikes                  24
5       Mountain Bikes                  60
6           Road Bikes                  60

In [22]:
SELECT category_name, COUNT(product_id) as 'Number of Products'
FROM products
JOIN categories ON products.category_id = categories.category_id
GROUP BY category_name;


category_name  Number of Products
0    Children Bicycles                  59
1     Comfort Bicycles                  30
2    Cruisers Bicycles                  78
3  Cyclocross Bicycles                  10
4       Electric Bikes                  24
5       Mountain Bikes                  60
6           Road Bikes                  60

### 3.Top three brands
This query will join the products and brands tables on the brand_id column. Then, it will group the data by brand_id and find the average list_price for each brand. Finally, it will order the data by avg_price in descending order and limit the result to 3 rows.


In [23]:
WITH brand_avg_prices AS (
SELECT brand_id, AVG(list_price) AS avg_price, ROW_NUMBER() OVER (ORDER BY AVG(list_price) DESC) as row_number
FROM products
GROUP BY brand_id
)
SELECT brand_name, avg_price
FROM brand_avg_prices
JOIN brands ON brand_avg_prices.brand_id = brands.brand_id
WHERE row_number <= 3
ORDER BY avg_price DESC;

brand_name    avg_price
0       Trek  2500.064074
1     Heller  2172.996666
2      Surly  1331.753600

### 4.Top three Categores
This query is similar to the one for finding the number of different products for each category, but it orders the data by num_products in descending order and limits the result to 3 rows.



In [24]:
WITH categories_num_products AS (
SELECT categories.category_id, categories.category_name, COUNT(DISTINCT products.product_id) AS num_products
FROM products
JOIN categories ON products.category_id = categories.category_id
GROUP BY categories.category_id, categories.category_name
), top_3_categories AS (
SELECT category_id, category_name, num_products, ROW_NUMBER() OVER (ORDER BY num_products DESC) as row_number
FROM categories_num_products
)
SELECT category_id, category_name, num_products
FROM top_3_categories
WHERE row_number <= 3
ORDER BY num_products DESC;

category_id      category_name  num_products
0            3  Cruisers Bicycles            78
1            6     Mountain Bikes            60
2            7         Road Bikes            60

### 5.Other Insights

Number of different products in each category:


In [25]:
SELECT
category_id,
COUNT(DISTINCT product_id) AS num_products
FROM products
GROUP BY category_id;

category_id  num_products
0            1            59
1            2            30
2            3            78
3            4            10
4            5            24
5            6            60
6            7            60

In [26]:
SELECT
category_id,
COUNT(DISTINCT product_id) AS num_products
FROM products
GROUP BY category_id;

category_id  num_products
0            1            59
1            2            30
2            3            78
3            4            10
4            5            24
5            6            60
6            7            60

Popularity of each category based on list price:


In [27]:
SELECT
category_name,
AVG(list_price) AS avg_list_price
FROM products
JOIN categories ON products.category_id = categories.category_id
GROUP BY category_name
ORDER BY avg_list_price DESC;

category_name  avg_list_price
0       Electric Bikes     3281.656666
1           Road Bikes     3175.357333
2  Cyclocross Bicycles     2542.793000
3       Mountain Bikes     1649.757333
4    Cruisers Bicycles      730.412307
5     Comfort Bicycles      682.123333
6    Children Bicycles      287.786610

In [28]:
SELECT
category_name,
AVG(list_price) AS avg_list_price
FROM products
JOIN categories ON products.category_id = categories.category_id
GROUP BY category_name
ORDER BY avg_list_price DESC;

category_name  avg_list_price
0       Electric Bikes     3281.656666
1           Road Bikes     3175.357333
2  Cyclocross Bicycles     2542.793000
3       Mountain Bikes     1649.757333
4    Cruisers Bicycles      730.412307
5     Comfort Bicycles      682.123333
6    Children Bicycles      287.786610

## Conclusion
This report analyzed the data available at a chain of bicycle stores, including information on the products, brands, and categories of items sold. The analysis aimed to provide insights into the products sold, including the most and least expensive items, the number of different products in each category, and the top three brands and categories with the highest average list price. The results of the analysis showed that Trek was the top brand with the highest average list price, while Road Bikes was the top category with the most number of different products. The report provides valuable information that can be used to make data-driven decisions in the bicycle market